In [6]:
import pandas as pd
import numpy as np

En dataset homicidios:
- Hacer rangos etarios
- Quitar la columna participantes
- Revisar la hora
- Quedarse solo con Tipo de calle, calle, comuna, y coordenadas
- Tratar de rellenar las coordenadas usando los points - o revisar si powerBI lee datos tipo point
- Revisar categorias de rol_victima

En dataset lesiones: - Por ahora no dedicarle mucho, porque tal vez no lo usemos
- Revisar horas y fechas
- Sacar dia de la semana
- Eliminar dirección, otra_dirección, altura, cruce, las columnas de moto, tte publico, etc
- Tratar de rellenar las coordenadas usando los points
- Estandarizar la columna sexo
- Imputar nulos en columna gravedad con gravedad leve

-- Final: Estandarizar las dos tablas con sus victimas y tratar de hacer un append. Añadir columna de lesion/homicidio o gravedad.

In [102]:
# Read datasets homicidios
h_hechos = pd.read_excel('datasets/homicidios.xlsx', sheet_name='HECHOS', na_values=['SD','SD ''SD-SD', 'Point (. .)', '.', ''])
h_victims = pd.read_excel('datasets/homicidios.xlsx', sheet_name='VICTIMAS', na_values=['SD', 'SD ','SD-SD'])

#Drop unncesary columns
h_hechos.drop(columns=['AAAA', 'MM', 'DD','HORA', 'LUGAR_DEL_HECHO', 'Altura', 'Cruce', 
                        'Dirección Normalizada', 'PARTICIPANTES','VICTIMA'], inplace=True)
h_victims.drop(columns=['FECHA', 'AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO'], inplace=True)

# Rename columns
h_hechos.rename(columns={'ID': 'siniestro_id','N_VICTIMAS':'num_victimas', 'FECHA':'fecha', 'HH':'hora', 'Calle':'calle', 
                        'COMUNA':'comuna', 'XY (CABA)': 'geocodificacion_caba', 'TIPO_DE_CALLE':'tipo_calle',
                        'pos x':'longitud', 'pos y':'latitud', 'VICTIMA':'vehiculo_victima', 'ACUSADO':'vehiculo_acusado'}, inplace=True)
h_victims.rename(columns={'ID_hecho':'siniestro_id', 'ROL':'rol_victima', 'VICTIMA':'vehiculo_victima', 'SEXO':'sexo_victima', 
                            'EDAD': 'edad_victima'}, inplace=True)

# Change to lowercase
h_hechos = h_hechos.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
h_victims = h_victims.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

# Change 'ciclista' for 'conductor' in victims 'rol' column
h_victims['rol_victima'] = h_victims['rol_victima'].apply(lambda x: 'conductor' if x == 'ciclista' else x)

# Categorize the ages in the victims table
bins = [0, 18, 29, 39, 49, 59, float('inf')]
labels = ['menos de 18', 'entre 18 y 29', 'entre 30 y 39', 'entre 40 y 49', 'entre 50 y 59', '60 y mayores']
h_victims['rango_etario'] = pd.cut(h_victims['edad_victima'], bins=bins, labels=labels, right=False)
h_victims.drop(columns=['edad_victima'], inplace=True)

# Add a column called 'gravedad' to differentiate from lesiones, once it's concatenated
h_victims.insert(loc=5, column='gravedad', value='fatal')

# Change dtype of lat and long
h_hechos['latitud'] = h_hechos['latitud'].astype('str')
h_hechos['longitud'] = h_hechos['longitud'].astype('str')

# Save to csv file
h_hechos.to_csv('datasets/homicidios_hechos.csv', index=False)
h_victims.to_csv('datasets/homicidios_victimas.csv', index=False)

In [27]:
# Read datasets lesiones
l_hechos = pd.read_excel('datasets/lesiones.xlsx', sheet_name='HECHOS', na_values=['SD','SD ','SD-SD','sd', 'Point (. .)', '.', '', 'No Especificada','no especificada'])
l_victims = pd.read_excel('datasets/lesiones.xlsx', sheet_name='VICTIMAS', na_values=['SD','SD ','sd', 'SD-SD', 'No Especificada','no especificada'])

#Drop unncesary columns
l_hechos.drop(columns=['aaaa', 'mm', 'dd','hora', 'direccion_normalizada', 'otra_direccion', 'altura', 'cruce', 
                        'participantes','victima', 'moto', 'auto', 'transporte_publico', 'camion', 'ciclista', 'gravedad'], inplace=True)
l_victims.drop(columns=['FECHA ', 'AAA', 'MM', 'DD'], inplace=True)

# Rename columns
l_hechos.rename(columns={'id': 'siniestro_id','n_victimas':'num_victimas', 'franja_hora':'hora', 
                        'COMUNA':'comuna', 'geocodificacion_CABA': 'geocodificacion_caba', 'latutid':'latitud', 'acusado':'vehiculo_acusado'}, inplace=True)
l_victims.rename(columns={'ID hecho':'siniestro_id', 'VEHICULO_VICTIMA':'vehiculo_victima', 'SEXO':'sexo_victima', 
                            'EDAD_VICTIMA': 'edad_victima', 'GRAVEDAD':'gravedad'}, inplace=True)

# Change to lowercase
l_hechos = l_hechos.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
l_victims = l_victims.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

# Change the dtype of edad_victima and fecha
l_victims['edad_victima'] = l_victims['edad_victima'].astype('Int64')
l_hechos['fecha'] = pd.to_datetime(l_hechos['fecha'], errors='coerce')

# Change dtype of lat and long
l_hechos['latitud'] = l_hechos['latitud'].astype('str')
l_hechos['longitud'] = l_hechos['longitud'].astype('str')

# Replace the nulls in column gravedad with 'leve' as stated in the data dictionary
l_victims['gravedad'].fillna('leve', inplace=True)

# Categorize the ages in the victims table
bins = [0, 18, 29, 39, 49, 59, float('inf')]
labels = ['menos de 18', 'entre 18 y 29', 'entre 30 y 39', 'entre 40 y 49', 'entre 50 y 59', '60 y mayores']
l_victims['rango_etario'] = pd.cut(l_victims['edad_victima'], bins=bins, labels=labels, right=False)
l_victims.drop(columns=['edad_victima'], inplace=True)

# Replace variable names in column sexo_victima to match homocidios
l_victims['sexo_victima'] = l_victims['sexo_victima'].apply(lambda x: 'femenino' if x=='mujer' else ( 'masculino' if x=='varon' else x))

# Rearrange columns order so it matches the ones in homicidios
cols_l_hechos = list(l_hechos.columns)
cols_l_hechos.insert(6, cols_l_hechos.pop(4))
l_hechos = l_hechos[cols_l_hechos]

cols_l_victims = list(l_victims.columns)
cols_l_victims.insert(3,cols_l_victims.pop(4))
l_victims = l_victims[cols_l_victims]
l_victims.insert(loc=1, column='rol_victima', value=np.nan)

# Save to csv file
l_hechos.to_csv('datasets/lesiones_hechos.csv', index=False)
l_victims.to_csv('datasets/lesiones_victimas.csv', index=False)

In [25]:
l_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   siniestro_id          23785 non-null  object        
 1   num_victimas          23785 non-null  int64         
 2   fecha                 23785 non-null  datetime64[ns]
 3   hora                  23776 non-null  float64       
 4   tipo_calle            12740 non-null  object        
 5   calle                 12867 non-null  object        
 6   comuna                22770 non-null  object        
 7   geocodificacion_caba  22533 non-null  object        
 8   longitud              23785 non-null  object        
 9   latitud               23785 non-null  object        
 10  vehiculo_acusado      8497 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(8)
memory usage: 2.0+ MB


In [23]:
h_hechos = pd.read_excel('datasets/homicidios.xlsx', sheet_name='HECHOS', na_values=['SD','SD ''SD-SD', 'Point (. .)', '.', ''])
h_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    float64       
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc